# Verify noise level for 5 noise cases
- 10\%
- 20\%
- STD from data
- avg STD from data
- avg over time from data

In [ ]:
# imports
import numpy as np
import cuqi
import sys
import matplotlib.pyplot as plt
from cuqi.distribution import Gaussian, JointDistribution
from cuqi.geometry import Continuous2D
from cuqi.pde import TimeDependentLinearPDE
from cuqi.model import PDEModel
from advection_diffusion_inference_utils import parse_commandline_args,\
    read_data_files,\
    create_domain_geometry,\
    create_PDE_form,\
    create_prior_distribution,\
    create_exact_solution_and_data,\
    set_the_noise_std,\
    sample_the_posterior,\
    create_experiment_tag,\
    plot_experiment,\
    save_experiment_data,\
    Args,\
    build_grids,\
    create_time_steps,\
    plot_time_series

#### Set up run arguments

In [ ]:
args = Args()
noise_level_list= ["from_data_var" , "from_data_avg", "avg_over_time", 0.1, 0.2]
args.noise_level = noise_level_list[2]
args.animal = 'm1'
args.ear = 'l'
args.inference_type = 'heterogeneous'


### Read the data

In [ ]:
times, locations, real_data, real_std_data = read_data_files(args)
# The left boundary condition is given by the data  
real_bc = real_data.reshape([len(locations), len(times)])[0,:]

### Create the forward model

In [ ]:
#%% STEP 4: Create the PDE grid and coefficients grid
#----------------------------------------------------
# PDE and coefficients grids
L = locations[-1]*1.3
coarsening_factor = 5
n_grid_c = 20
grid, grid_c, grid_c_fine, h, n_grid = build_grids(L, coarsening_factor, n_grid_c)

#%% STEP 5: Create the PDE time steps array
#------------------------------------------
tau_max = 30*60 # Final time in sec
cfl = 5 # The cfl condition to have a stable solution
         # the method is implicit, we can choose relatively large time steps 
tau = create_time_steps(h, cfl, tau_max)

#%% STEP 6: Create the domain geometry
#-------------------------------------
G_c = create_domain_geometry(grid_c, args.inference_type)

# STEP 7: Create the PDE form
#----------------------------
PDE_form = create_PDE_form(real_bc, grid, grid_c, grid_c_fine, n_grid, h, times,
                           args.inference_type)

# STEP 8: Create the CUQIpy PDE object
#-------------------------------------
PDE = TimeDependentLinearPDE(PDE_form,
                             tau,
                             grid_sol=grid,
                             method='backward_euler', 
                             grid_obs=locations,
                             time_obs=times) 

# STEP 9: Create the range geometry
#----------------------------------
G_cont2D = Continuous2D((locations, times))

# STEP 10: Create the CUQIpy PDE model
#-------------------------------------
A = PDEModel(PDE, range_geometry=G_cont2D, domain_geometry=G_c)

#### Create the prior distribution

In [ ]:
# STEP 11: Create the prior distribution
#---------------------------------------
x = create_prior_distribution(G_c, args.inference_type)

#from cuqi.distribution import GMRF
#x = GMRF(np.ones(G_c.par_dim)*np.sqrt(300), 0.2, geometry=G_c, bc_type='neumann')

Ns = 10
samples_x = x.sample(Ns)
samples_x.plot(range(Ns), plot_par=True)
plt.figure()
samples_x.funvals.plot(range(Ns))

#### Create and plot exact data

In [ ]:
x_true = np.ones(x.dim)*100
exact_data = A(x_true, is_par=False)
plot_time_series(times, locations, exact_data.reshape([len(locations), len(times)]))


#### Create the data distribution

In [ ]:
#%% STEP 13: Create the data distribution
#----------------------------------------
# First, illustrate how different noise levels setups affect the std
print("Standard deviation values")
plt.figure()
for item in noise_level_list:
    s_noise_temp = set_the_noise_std(
        args.data_type, item, exact_data,
        None, real_std_data, G_cont2D)
    print('\n**noise_level option**:', item)
    print('**STD of the noise**:')
    print(s_noise_temp)
    # plot noise level
    if isinstance(s_noise_temp, np.ndarray):
        plt.plot(s_noise_temp, label=str(item))
    else:
        plt.plot(s_noise_temp*np.ones(G_cont2D.par_dim), label=str(item))
plt.legend()
plt.title('Noise level')
plt.xlabel('data point index i\n data point for the same location are grouped together')
plt.ylabel('std of the noise')

# Second, set the noise level
s_noise = set_the_noise_std(args.data_type, args.noise_level, exact_data,
                                None, real_std_data, G_cont2D)
y = Gaussian(A(x), s_noise**2, geometry=G_cont2D)

#### Samples of noisy data and plot the noisy data and the noise

In [ ]:
noisy_data = y(x=x_true).sample()

fig, ax = plt.subplots(1, 3, figsize=(14, 3))
plt.sca(ax[0])
plot_time_series(times, locations, exact_data.reshape([len(locations), len(times)]))
plt.title('Exact data')

plt.sca(ax[1])
plot_time_series(times, locations, noisy_data.reshape([len(locations), len(times)]))
plt.title('Noisy data')

diff = noisy_data - exact_data
plt.sca(ax[2])
plot_time_series(times, locations, diff.reshape([len(locations), len(times)]))
plt.title('Difference')
# Noise to signal ratio
print('Noise to signal ratio (synthitic data): ', np.linalg.norm(diff)/np.linalg.norm(noisy_data))
print('STD to signal ratio (real data): ', np.linalg.norm(real_std_data)/np.linalg.norm(real_data))


#### How subsampling the data affects the noise level

In [ ]:
# assume we have n observations of the same value
# The values is 7
sl = [2, 4, 8]
means = []
stds = []

ns = [40*k for k in range(1,10)]
stds_sub = np.empty((len(sl), len(ns)))
for i, n in enumerate(ns):
  value = 7
  obs = value + np.random.randn(n)*0.1
  for j, subsample_level in enumerate(sl):
    # average the observations
    obs_sub = np.mean(obs.reshape(-1, subsample_level), axis=1)
    stds_sub[j, i] = np.std(obs_sub)

  means.append(np.mean(obs))
  stds.append(np.std(obs))

fig, ax = plt.subplots(1, 2, figsize=(12, 3))
plt.sca(ax[0])
plt.plot(ns, means, 'o', label='mean')
plt.plot(ns,[value]*len(means), 'r--', label='true value')
plt.legend()
plt.sca(ax[1])
plt.plot(ns, stds, 'o', label='std')
for l in range(len(sl)):
  plt.plot(ns, stds_sub[l], 'o', label='std subsample'+str(sl[l]))
plt.plot(ns, [0.1]*len(stds), 'r--', label='true std')
plt.legend()